In [88]:
import sys
import idx2numpy
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [89]:
import sys
import numpy as np
import pandas as pd
import torch

def one_hot(label):
    # 用于把标签变成one-hot格式
    m = len(label)
    label_one_hot = torch.zeros([m,10])
    for i in range(m):
        label_one_hot[i,label[i]] = 1
    return label_one_hot

def preprocess(image):
    image_new = image/255
    return image_new

def rand_indices():
    indices = torch.randperm(20000)
    train_size, validation_size = 5000, 5000

    train_indices = indices[:train_size]
    validation_indices = indices[train_size:(train_size + validation_size)]
    test_indices = indices[(train_size + validation_size):]
    return train_indices, validation_indices, test_indices

def pollute_label(label):
    indices = torch.randperm(5000)
    polluted_indices = indices[:2500]
    for i in polluted_indices:
        label[i] = torch.zeros(10).scatter_(0, torch.randint(10, (1,)), 1)
    return label

def load_data_from_csv(flag):
    N = 20000
    data = pd.read_csv('mnist/train.csv').to_numpy()
    data = torch.from_numpy(data)
    image_total = data[1:N+1,1:]
    label_total = one_hot(data[:,0])

    tr_ind, val_ind, test_ind = rand_indices()
    image_train = preprocess(image_total[tr_ind])
    image_validation = preprocess(image_total[val_ind])
    image_test = preprocess(image_total[test_ind])

    if flag==1:
        label_train = pollute_label(label_total[tr_ind]).float()
    if flag==0:
        label_train = label_total[tr_ind].float()
    label_validation = label_total[val_ind].float()
    label_test = label_total[test_ind].float()
    return (image_train, label_train), (image_validation, label_validation), (image_test, label_test)


In [90]:
data_tr,data_val,data_test = load_data_from_csv(1)

In [91]:
p = 28
C = 10
N = 5000

img_tr, lab_tr = data_tr
img_val, lab_val = data_val
img_test, lab_test = data_test

## 定义神经网络
#W = torch.rand([p**2,C],requires_grad=True)
#b = torch.rand([N,C],requires_grad=True)
la = torch.rand([N,1],requires_grad=True)

class Inner_loop(nn.Module):
    def __init__(self):
        super(Inner_loop, self).__init__()
        self.fc = nn.Linear(28*28, 10)

    def forward(self, x):
        x = self.fc(x)
        return x

net = Inner_loop()

# 定义损失函数和优化器(以后的函数均只用一组数据，然后累加)

def loss_fun1(outputs, labels):
    crossentropy = nn.CrossEntropyLoss()
    loss = crossentropy(outputs, labels)
    return loss

def loss_fun2(outputs, labels):
    crossentropy = nn.CrossEntropyLoss()
    loss = crossentropy(outputs, labels)
    return loss

optimizer = optim.Adam(net.parameters(), lr=1e-2)
optimizer1 = optim.Adam([la], lr=1e-2)

# 训练网络

for j in range(50):
    running_loss = 0.0
    images = img_tr
    labels = lab_tr
    for i, image in enumerate(images, 0):
        label = labels[i]

        optimizer.zero_grad()
        
        hat_label = net(image)
        loss = loss_fun1(hat_label, label)*la[i]
        running_loss += loss
        loss.backward()
        optimizer.step()

    if j % 10 == 0:
        print(f'loss1={running_loss/images.shape[0]},times={j}')
        
running_loss1 = 0.0
images1 = img_val
labels1 = lab_val
for i, image in enumerate(images1, 0):
    label = labels1[i]
    hat_label = net(image)
    loss1 = loss_fun2(hat_label, label)
    running_loss += loss
    loss1.backward()
    
    optimizer1.step()
    
print(f'loss2={running_loss1/images.shape[0]},times={1}')

# 测试网络
correct = 0
total = img_test.shape[0]
with torch.no_grad():
    for i, image in enumerate(img_test, 0):
        label = lab_test[i]
        outputs = net(image)
        
        _, predicted = torch.max(outputs, 0)
        _, labels = torch.max(label,0)
        correct += (predicted == labels).sum()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

loss1=tensor([2.2743], grad_fn=<DivBackward0>),times=0
loss1=tensor([1.9556], grad_fn=<DivBackward0>),times=10
loss1=tensor([1.9112], grad_fn=<DivBackward0>),times=20
loss1=tensor([1.9134], grad_fn=<DivBackward0>),times=30
loss1=tensor([1.9126], grad_fn=<DivBackward0>),times=40
loss2=0.0,times=1
Accuracy of the network on the 10000 test images: 10.180000305175781 %
